In [83]:
import pandas as pd

In [84]:
columns_name = ['FL_DATE', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 
                'CANCELLED', 'DIVERTED', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 
                'DISTANCE']

iata_code = ['PHX', 'ABQ', 'DEN', 'SAT', 'DFW', 'IAH', 'MCI', 'MSP', 'STL', 'ORD', 'BNA', 'IND', 'ATL', 'DTW', 'JAX', 'CLT', 
             'MIA', 'PIT', 'YYZ', 'PHL', 'JFK', 'LGA', 'YUL', 'BOS', 'WAW', 'TLV', 'ETH', 'HFA', 'TLV']

columns_time = ['CRS_DEP_TIME', 'DEP_TIME']

city_iata = {'Phoenix': 'PHX', 'Albuquerque': 'ABQ','Denver': 'DEN', 'San Antonio':'SAT', 'Dallas': 'DFW', 'Houston': 'IAH', 
             'Kansas City': 'MCI', 'Minneapolis': 'MSP', 'Saint Louis': 'STL', 'Chicago': 'ORD', 'Nashville': 'BNA', 
             'Indianapolis': 'IND', 'Atlanta': 'ATL', 'Detroit': 'DTW', 'Jacksonville': 'JAX', 'Charlotte': 'CLT', 
             'Miami': 'MIA', 'Pittsburgh': 'PIT', 'Toronto': 'YYZ', 'Philadelphia': 'PHL', 'New York': 'JFK', 'Montreal': 'YUL',
             'Boston': 'BOS'}

columns = ['datetime', 'Phoenix', 'Albuquerque', 'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City', 'Minneapolis', 'Saint Louis', 'Chicago', 
           'Nashville', 'Indianapolis', 'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami', 'Pittsburgh', 'Toronto', 
           'Philadelphia', 'New York', 'Montreal', 'Boston']

In [85]:
def format_time(value):
    value = str(value).replace('.0', '')[:4]
    # Преобразуем в строку и заполняем нулями слева, если нужно
    if len(value) == 1:
        return f"00:0{value}"
    elif len(value) == 2:
        return f"00:{value}"
    elif len(value) == 3:
        return f"{value[0]}:{value[1:]}"
    else:
        return f"{value[:2]}:{value[2:]}"

In [86]:
flights_2017 = pd.read_csv('.'
                           '/datasets/2017.csv', usecols=columns_name)
flights_2016 = pd.read_csv('./datasets/2016.csv', usecols=columns_name)
flights_2015 = pd.read_csv('./datasets/2015.csv', usecols=columns_name)
flights_2014 = pd.read_csv('./datasets/2014.csv', usecols=columns_name)
flights_2013 = pd.read_csv('./datasets/2013.csv', usecols=columns_name)
flights_2012 = pd.read_csv('./datasets/2012.csv', usecols=columns_name)

In [87]:
dataframes = [flights_2017, flights_2016, flights_2015, flights_2014, flights_2013, flights_2012]

In [88]:
humidity = pd.read_csv('./datasets/humidity.csv', usecols=columns)
pressure = pd.read_csv('./datasets/pressure.csv', usecols=columns)
temperature = pd.read_csv('./datasets/temperature.csv', usecols=columns)
wind_speed = pd.read_csv('./datasets/wind_speed.csv', usecols=columns)

In [89]:
humidity['datetime'] = pd.to_datetime(humidity['datetime'])
pressure['datetime'] = pd.to_datetime(pressure['datetime'])
temperature['datetime'] = pd.to_datetime(temperature['datetime'])
wind_speed['datetime'] = pd.to_datetime(wind_speed['datetime'])

In [90]:
humidity = humidity.melt(id_vars=["datetime"], var_name="ORIGIN", value_name="humidity")
pressure = pressure.melt(id_vars=["datetime"], var_name="ORIGIN", value_name="pressure")
temperature = temperature.melt(id_vars=["datetime"], var_name="ORIGIN", value_name="temperature")
wind_speed = wind_speed.melt(id_vars=["datetime"], var_name="ORIGIN", value_name="wind_speed")

weather = pd.concat([humidity, pressure, temperature, wind_speed], axis=1)
weather = weather.loc[:, ~weather.columns.duplicated()]
weather['ORIGIN'] = weather['ORIGIN'].map(city_iata)

In [91]:
weather['avg_humidity'] = weather['humidity'].rolling(window=3, min_periods=1).mean()
weather['avg_pressure'] = weather['pressure'].rolling(window=3, min_periods=1).mean()
weather['avg_temperature'] = weather['temperature'].rolling(window=3, min_periods=1).mean()
weather['avg_wind_speed'] = weather['wind_speed'].rolling(window=3, min_periods=1).mean()

In [92]:
def preprocess_data(df):
    df = df[(df['ORIGIN'].isin(iata_code)) & (df['DEST'].isin(iata_code))]
    #df.dropna(subset='DEP_DELAY', axis=0, inplace=True)
    df.loc[df["CANCELLED"] == 1, "DEP_DELAY"] = -999
    
    impute_column = ['DEP_TIME','CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME']
    for column in impute_column:
        df[column].fillna(0, inplace=True) 
    
    for column in columns_time:
        df[column] = df[column].apply(format_time)
        df[column] = df[column].str[:5]
    
    df['CRS_DEP_TIME'] = df['CRS_DEP_TIME'].str.replace('24:00', '00:00')
    df['DEP_TIME'] = df['DEP_TIME'].str.replace('24:00', '00:00')
   
    df['CRS_DEP_TIME'] = df['FL_DATE'] + ' ' + df['CRS_DEP_TIME'] 
    df['DEP_TIME'] = df['FL_DATE'] + ' ' + df['DEP_TIME']

    df['CRS_DEP_TIME'] = pd.to_datetime(df['CRS_DEP_TIME'])
    df['DEP_TIME'] = pd.to_datetime(df['DEP_TIME'])

    #df.dropna(inplace=True)
    
    # Объединяем по ORIGIN и DEP_TIME, берём ближайшую предшествующую влажность (не старше 3 часов)
    df_flights = pd.merge_asof(
        df.sort_values("CRS_DEP_TIME"),  # Обязательно сортируем!
        weather.sort_values("datetime"),
        left_on="CRS_DEP_TIME",
        right_on="datetime",
        by="ORIGIN",
        direction="backward",  # Берём предыдущее значение влажности
        tolerance=pd.Timedelta(hours=3)  # Ограничиваем 3 часами
    )
    
    drop_columns = ['datetime', 'humidity', 'pressure', 'temperature', 'wind_speed', 'FL_DATE', 'DIVERTED', 'DEP_TIME', 
                    'ORIGIN', 'DEST']
    
    df_flights.drop(drop_columns, axis=1, inplace=True)
    #df_flights.dropna(subset=['avg_humidity', 'avg_pressure', 'avg_temperature', 'avg_wind_speed'], inplace=True)
    
    df_flights['avg_pressure'] = round(df_flights['avg_pressure'] / 1.33322)
    df_flights['avg_temperature'] = round(df_flights['avg_temperature'] - 273.15)
    
    seasons = {1: 'winter', 2: 'spring', 3: 'summer', 4: 'autumn'}
    # Извлекаем год, месяц, день, день недели, час, четверть
    df_flights["year"] = df_flights["CRS_DEP_TIME"].dt.year
    df_flights["month"] = df_flights["CRS_DEP_TIME"].dt.month
    df_flights["day"] = df_flights["CRS_DEP_TIME"].dt.day
    df_flights["weekday"] = df_flights["CRS_DEP_TIME"].dt.weekday  # 0 - понедельник, 6 - воскресенье
    df_flights["hour"] = df_flights["CRS_DEP_TIME"].dt.hour
    df_flights["seasons"] = df_flights["CRS_DEP_TIME"].dt.quarter.map(seasons)

    # Выделяем период суток
    df_flights["time_of_day"] = pd.cut(df_flights["hour"], bins=[-1, 5, 9, 16, 24], labels=["night", "morning", "afternoon", "evening"])

    # Дополнительно можно добавить признак, связанный с выходными днями
    df_flights["weekend"] = df_flights["weekday"].isin([5, 6])  # 5 - суббота, 6 - воскресенье
    
    return df_flights


In [93]:
processed_dfs = [preprocess_data(df) for df in dataframes]
# flights_2017.isna().sum()

C:\Users\NIKITOS\AppData\Local\Temp\ipykernel_80488\2532787936.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(0, inplace=True)
C:\Users\NIKITOS\AppData\Local\Temp\ipykernel_80488\2532787936.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].fillna(0, inplace=True)
C:\Users\NIKITOS\AppData\Local\Temp\ipykernel

In [94]:
fligths = pd.concat(processed_dfs)
flights = pd.get_dummies(fligths, columns=['seasons', 'time_of_day'])

,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,avg_humidity,avg_pressure,...,hour,weekend,seasons_autumn,seasons_spring,seasons_summer,seasons_winter,time_of_day_night,time_of_day_morning,time_of_day_afternoon,time_of_day_evening
131,2017-01-01 06:30:00,-999.0,NaN,1.0,355.0,0.0,0.0,2153.0,45.000000,760.0,...,6,True,False,False,False,True,False,True,False,False
590,2017-01-01 09:57:00,-999.0,NaN,1.0,296.0,0.0,0.0,2153.0,95.666667,758.0,...,9,True,False,False,False,True,False,True,False,False
829,2017-01-01 11:45:00,-999.0,NaN,1.0,110.0,0.0,0.0,449.0,54.666667,761.0,...,11,True,False,False,False,True,False,False,True,False
1217,2017-01-01 14:49:00,-999.0,NaN,1.0,153.0,0.0,0.0,907.0,97.666667,764.0,...,14,True,False,False,False,True,False,False,True,False
1442,2017-01-01 16:22:00,-999.0,NaN,1.0,69.0,0.0,0.0,247.0,89.000000,760.0,...,16,True,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015958,2012-12-30 16:20:00,-999.0,NaN,1.0,240.0,0.0,0.0,1303.0,74.500000,757.0,...,16,True,True,False,False,False,False,False,True,False
1016037,2012-12-30 17:00:00,-999.0,NaN,1.0,75.0,0.0,0.0,184.0,NaN,NaN,...,17,True,True,False,False,False,False,False,False,True
1016530,2012-12-30 19:50:00,-999.0,NaN,1.0,72.0,0.0,0.0,328.0,43.666667,765.0,...,19,True,True,False,False,False,False,False,False,True
1016685,2012-12-30 21:40:00,-999.0,NaN,1.0,82.0,0.0,0.0,328.0,37.333333,768.0,...,21,True,True,False,False,False,False,False,False,True


In [95]:
flights.to_csv("flights_dataset.csv")

In [96]:
flights.isna().sum()

CRS_DEP_TIME                   0
DEP_DELAY                      0
TAXI_OUT                   87450
CANCELLED                      0
CRS_ELAPSED_TIME               0
ACTUAL_ELAPSED_TIME            0
AIR_TIME                       0
DISTANCE                       0
avg_humidity             1187752
avg_pressure             1170578
avg_temperature          1161891
avg_wind_speed           1161848
year                           0
month                          0
day                            0
weekday                        0
hour                           0
weekend                        0
seasons_autumn                 0
seasons_spring                 0
seasons_summer                 0
seasons_winter                 0
time_of_day_night              0
time_of_day_morning            0
time_of_day_afternoon          0
time_of_day_evening            0
dtype: int64

In [97]:
datta = pd.read_csv("./datasets/flights_dataset.csv").drop(["Unnamed: 0"], axis=1)
datta

,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,avg_humidity,avg_pressure,...,hour,weekend,seasons_autumn,seasons_spring,seasons_summer,seasons_winter,time_of_day_night,time_of_day_morning,time_of_day_afternoon,time_of_day_evening
0,2017-01-01 00:35:00,-4.0,10.0,0.0,227.0,205.0,189.0,1773.0,73.666667,759.0,...,0,True,False,False,False,True,True,False,False,False
1,2017-01-01 00:45:00,-3.0,12.0,0.0,202.0,193.0,171.0,1558.0,39.333333,755.0,...,0,True,False,False,False,True,True,False,False,False
2,2017-01-01 01:55:00,-7.0,15.0,0.0,247.0,237.0,218.0,1972.0,75.333333,759.0,...,1,True,False,False,False,True,True,False,False,False
3,2017-01-01 02:05:00,-16.0,9.0,0.0,127.0,119.0,101.0,868.0,77.000000,759.0,...,2,True,False,False,False,True,True,False,False,False
4,2017-01-01 02:20:00,21.0,11.0,0.0,228.0,225.0,201.0,1773.0,77.000000,759.0,...,2,True,False,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4699312,2012-12-31 22:55:00,-1.0,13.0,0.0,64.0,59.0,40.0,187.0,41.500000,761.0,...,22,False,True,False,False,False,False,False,False,True
4699313,2012-12-31 22:55:00,-6.0,11.0,0.0,64.0,60.0,47.0,247.0,35.333333,771.0,...,22,False,True,False,False,False,False,False,False,True
4699314,2012-12-31 22:59:00,5.0,11.0,0.0,71.0,63.0,47.0,328.0,41.500000,759.0,...,22,False,True,False,False,False,False,False,False,True
4699315,2012-12-31 23:41:00,97.0,10.0,0.0,282.0,233.0,218.0,2300.0,40.000000,759.0,...,23,False,True,False,False,False,False,False,False,True
